In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm  # barra de progresso

### CONFIGURAÇÕES
MODEL_PATH = "./bert_final" #caminho do modelo local
BATCH_SIZE = 32 # qtd de textos processados por vez pelo bert
DEVICE = "cuda" if torch.cuda.is_available() else "cpu" # se não achar gpu, usa cpu

print(f"OK dispositivo detectado: {DEVICE.upper()}")
print(f"OK carregando modelo de: {MODEL_PATH} ...")

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH) # carrega tokenizer
model = AutoModel.from_pretrained(MODEL_PATH).to(DEVICE) # carrega modelo bert

# coloca o modelo em modo de avaliação (eval)
# isso desliga o cálculo de gradientes e camadas de dropout
# economiza memória e deixa o processo mais rápido
model.eval()

print("Modelo carregado\n")

In [ ]:
# CARREGAMENTO DOS DADOS
print("Carregando dados para treinamento e teste...")

# DADOS HISTÓRICOS (para aprendizado)
# n exemplos passados
# A coluna 'TARGET_FULL' concatena os 3 níveis para o modelo aprender tudo junto
dados_reais = [
    # --- FAMÍLIA: CARTÕES ---
    ("Não reconheço essa compra na padaria", "Cartão | Contestação | Compra Não Reconhecida"),
    ("Apareceu uma assinatura de streaming que eu cancelei", "Cartão | Contestação | Compra Recorrente"),
    ("Clonaram meu cartão, tem gastos em dolar", "Cartão | Contestação | Fraude Externa"),
    ("Minha fatura veio com anuidade sendo que era isento", "Cartão | Tarifas | Cobrança de Anuidade"),
    ("Quero aumentar meu limite para viajar", "Cartão | Limite | Análise de Crédito"),
    ("O cartão chegou mas a senha não funciona", "Cartão | Entrega | Desbloqueio/Senha"),
    ("Recebi um cartão que eu nunca pedi", "Cartão | Contratação | Cartão não Solicitado"),

    # --- FAMÍLIA: CONTA CORRENTE / PIX ---
    ("Meu pix saiu da conta mas a loja diz que não recebeu", "Conta Corrente | Pix | Falha Tecnológica"),
    ("Fiz um pix errado, quero o dinheiro de volta", "Conta Corrente | Pix | Devolução (MED)"),
    ("Tem uma tarifa de cesta de serviços que não contratei", "Conta Corrente | Tarifas | Pacote de Serviços"),
    ("Não consigo acessar minha conta pelo celular", "Canais Digitais | App | Erro de Acesso"),
    ("O app fecha sozinho quando tento pagar boleto", "Canais Digitais | App | Instabilidade"),
    ("Quero encerrar minha conta, vou mudar de banco", "Conta Corrente | Encerramento | Cancelamento Voluntário"),
    ("Vocês bloquearam minha conta sem avisar", "Conta Corrente | Bloqueio | Segurança/Compliance"),

    # --- FAMÍLIA: EMPRÉSTIMOS / COBRANÇA ---
    ("Os juros desse empréstimo estão abusivos", "Empréstimo | Pessoal | Revisão de Juros"),
    ("Quero quitar meu financiamento de carro antecipado", "Empréstimo | Veículos | Amortização"),
    ("Estou recebendo ligações de cobrança o dia todo", "Cobrança | Abordagem | Excesso de Ligações"),
    ("Quero renegociar minha dívida, estou desempregado", "Cobrança | Renegociação | Acordo"),
    ("Descontaram a parcela do emprestimo duas vezes", "Empréstimo | Consignado | Desconto Indevido"),

    # --- FAMÍLIA: ATENDIMENTO ---
    ("O gerente foi extremamente grosso comigo", "Atendimento | Agência | Postura do Funcionário"),
    ("Fiquei 2 horas na fila esperando atendimento", "Atendimento | Agência | Tempo de Espera"),
    ("O SAC desliga a chamada na minha cara", "Atendimento | SAC | Qualidade da Ligação"),
    ("Ninguém resolve meu problema, ficam jogando de um para o outro", "Atendimento | Geral | Falta de Resolução"),

    # --- FAMÍLIA: INVESTIMENTOS ---
    ("Quero resgatar meu dinheiro da poupança e não consigo", "Investimentos | Poupança | Resgate"),
    ("Meu assessor de investimentos me recomendou algo que deu prejuízo", "Investimentos | Assessoria | Suitability"),
    ("Não recebi o informe de rendimentos para o imposto de renda", "Investimentos | Documentação | Informe de Rendimentos")
]

# Criando o DataFrame
df_historico = pd.DataFrame(dados_reais, columns=['TEXTO_CLIENTE', 'TARGET_FULL'])

# DADOS NOVOS (para classificar)
# Estes são os 40.000 que você quer classificar. Simulando 5 exemplos aqui.
df_novos = pd.DataFrame({
    'ID_MANIFESTACAO': range(1001, 1016), # IDs fictícios de 1001 a 1015
    'TEXTO_CLIENTE': [
        # --- Variações de CARTÃO (Treino: "Não reconheço compra") ---
        "Apareceu uma cobrança da Netflix na minha fatura que eu não assinei.",
        "Tem um débito de 500 reais do Mercado Livre que é fraude.",
        "Veio uma compra internacional que desconheço.",

        # --- Variações de PIX (Treino: "Pix saiu mas não chegou") ---
        "Fiz uma transferência instantânea pro meu irmão e o dinheiro sumiu.",
        "O comprovante do pix saiu, mas o destinatário diz que não recebeu nada.",
        "Erro na transação, debitou o saldo mas não completou o envio.",

        # --- Variações de CANCELAMENTO (Treino: "Quero cancelar conta") ---
        "Gostaria de encerrar meu relacionamento com este banco.",
        "Desejo fechar minha conta, não uso mais.",
        "Qual o procedimento para o encerramento da conta corrente?",

        # --- Variações de ATENDIMENTO (Treino: "Gerente mal educado") ---
        "O atendente do caixa foi extremamente grosso comigo hoje.",
        "Fui destratado na agência da Avenida Paulista.",
        "Falta de respeito por parte dos funcionários da agência.",

        # --- Variações de JUROS (Treino: "Juros abusivos") ---
        "A taxa que vocês estão cobrando no cheque especial é um absurdo.",
        "Descontaram uns encargos da minha conta que eu não concordo.",
        "O valor dos juros rotativos está muito alto este mês.",
        
        # --- CASO AMBÍGUO (Teste de Stress) ---
        "Vocês estão roubando meu dinheiro com essas taxas no cartão." 
        # (Mistura "Cartão" com "Taxas/Juros" - vamos ver o que ele decide)
    ]
})

In [ ]:
# FUNÇÃO DE GERAÇÃO DE VETORES (EMBEDDINGS)
def generate_embeddings_batched(texts, tokenizer, model, batch_size=32):
    """
    Função que recebe uma lista de textos e retorna uma matriz numérica
    Usa processamento em batches para não estourar a memória RAM
    """
    all_embeddings = []
    
    # barra de progresso no terminal
    for i in tqdm(range(0, len(texts), batch_size), desc="Vetorizando textos"):
        
        # pega uma fatia da lista de textos (ex: do 0 ao 32, do 32 ao 64...)
        batch_texts = texts[i:i + batch_size]
        
        # 1. Tokenização: Prepara o texto para o BERT
        # padding=True: Preenche frases curtas para ficarem do mesmo tamanho
        # truncation=True: Corta frases maiores que 128 tokens (limite comum)
        inputs = tokenizer(
            batch_texts, 
            padding=True, 
            truncation=True, 
            max_length=128, 
            return_tensors="pt" # retorna tensores do PyTorch
        ).to(DEVICE) # move os dados para CPU ou GPU
        
        # 2. Inferência: O texto passa pela rede neural
        with torch.no_grad(): # Desabilita cálculo de treino (economia gigantesca de RAM)
            outputs = model(**inputs)
            
        # 3. Pooling: Transformar o resultado complexo em um único vetor por frase
        # Pegamos a média (mean) da última camada escondida (last_hidden_state)
        # .cpu().numpy() traz os dados da placa de vídeo de volta para a memória RAM normal
        embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
        
        # Adiciona o resultado parcial na lista final
        all_embeddings.append(embeddings)
        
    # junta todos os pedaços em uma única matriz numpy gigante
    return np.vstack(all_embeddings)

In [ ]:
# TREINAMENTO DO MODELO (supervisionado)
print(f"\n[Fase 1] Processando {len(df_historico)} registros do histórico...")

# vetores para o histórico x
X_historico = generate_embeddings_batched(
    df_historico['TEXTO_CLIENTE'].tolist(), 
    tokenizer, 
    model, 
    BATCH_SIZE
)

# target y
y_historico = df_historico['TARGET_FULL']

# treino e validação
X_train, X_val, y_train, y_val = train_test_split(X_historico, y_historico, test_size=0.2, random_state=42)

# cria e treina o classificador (MLP - Multi Layer Perceptron)
# esta é a rede neural "leve" que vai aprender as regras do banco
print("\n[Fase 2] Treinando o classificador baseada no histórico...")
clf = MLPClassifier(
    hidden_layer_sizes=(256, 128), # camadas ocultas (neurônios)
    max_iter=300,                  # máximo de épocas de treino
    random_state=42,               # semente para reprodutibilidade
    verbose=True                   # mostra o log de loss no terminal
)

clf.fit(X_train, y_train) # treinamento

# avaliação
print("\n--- Validação do Modelo (Acurácia) ---")
print(f"Acurácia nos dados de teste: {clf.score(X_val, y_val):.2%}")

In [ ]:
# PREDIÇÃO NOS DADOS NOVOS (TOP 3)
print(f"\n[Fase 3] Classificando {len(df_novos)} novos casos...")

# vetores para os dados novos
X_novos = generate_embeddings_batched(
    df_novos['TEXTO_CLIENTE'].tolist(), 
    tokenizer, 
    model, 
    BATCH_SIZE
)

# obtém as probabilidades de TODAS as classes para cada texto
# retorna uma matriz onde linhas = textos e colunas = probabilidade de cada categoria
print("Calculando probabilidades...")
probas = clf.predict_proba(X_novos)

# lista de nomes das classes (na ordem das colunas da matriz probas)
classes = clf.classes_

# llistas para armazenar o resultado final
top1_label, top1_score = [], []
top2_label, top2_score = [], []
top3_label, top3_score = [], []

# loop para processar cada linha e extrair o Top 3
for i in tqdm(range(len(probas)), desc="Extraindo Top 3"):
    row_probas = probas[i]
    
    # argsort retorna os ÍNDICES ordenados do menor para o maior
    # Pegamos os últimos 3 ([-3:]) e invertemos ([: : -1]) para ter: 1º, 2º, 3º
    top_indices = np.argsort(row_probas)[-3:][::-1]
    
    # 1º lugar
    idx1 = top_indices[0]
    top1_label.append(classes[idx1])
    top1_score.append(row_probas[idx1])
    
    # 2º lugar (Verifica se existe, caso haja menos de 2 categorias no treino)
    if len(top_indices) > 1:
        idx2 = top_indices[1]
        top2_label.append(classes[idx2])
        top2_score.append(row_probas[idx2])
    else:
        top2_label.append(None)
        top2_score.append(0.0)
        
    # 3º lugar
    if len(top_indices) > 2:
        idx3 = top_indices[2]
        top3_label.append(classes[idx3])
        top3_score.append(row_probas[idx3])
    else:
        top3_label.append(None)
        top3_score.append(0.0)

In [ ]:
# DATAFRAME FINAL

# Adiciona as colunas no dataframe original de novos dados
df_novos['RECOMENDACAO_1'] = top1_label
df_novos['SCORE_1'] = top1_score

df_novos['RECOMENDACAO_2'] = top2_label
df_novos['SCORE_2'] = top2_score

df_novos['RECOMENDACAO_3'] = top3_label
df_novos['SCORE_3'] = top3_score

print("\n--- PROCESSO CONCLUÍDO! ---")
print("Visualizando as primeiras linhas do resultado:")

# Mostra as colunas principais
cols = ['TEXTO_CLIENTE', 'RECOMENDACAO_1', 'SCORE_1', 'RECOMENDACAO_2', 'SCORE_2']
df_novos[cols].head()